This notebook provides example code for creating a basic RAG system using the LangChain framework.

In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFacePipeline
from langchain.chains import RetrievalQA

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

/home/jchen/fine-tuning/LLM-systems-intro/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:

# Step 1: Load a PDF and extract text
pdf_path = "../data/div-class-title-relaxing-assumptions-improving-inference-integrating-machine-learning-and-the-linear-regression-div.pdf"  # TODO Replace with your PDF path
loader = PyPDFLoader(pdf_path)
pages = loader.load()

# Step 2: Split text into manageable chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = text_splitter.split_documents(pages)

# Step 3: Embed the documents using a HuggingFace model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Step 4: Store them in a FAISS vector store for retrieval
vectorstore = Chroma.from_documents(documents, embedding_model)

# Step 5: Load a local LLM from Hugging Face
model_id = "Gensyn/Qwen2.5-1.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16)

# Create a generation pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=300)

# Wrap in LangChain's HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=generator)

# Step 6: Set up Retrieval-Augmented Generation (RAG)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)


Device set to use cuda:0


In [9]:

# Step 7: Ask questions based on PDF content
query = "What is the abstract?"
print("Question: ", query)
result = qa_chain(query)

# Output result
print("Answer:")
print(result["result"])

Question:  What is the abstract?
Answer:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

full details in Appendix A of the Supplementary Materials.
Marc Ratkovic
1056
https://doi.org/10.1017/S0003055422001022
 Published online by Cambridge University Press

full details in Appendix A of the Supplementary Materials.
Marc Ratkovic
1056
https://doi.org/10.1017/S0003055422001022
 Published online by Cambridge University Press

full details in Appendix A of the Supplementary Materials.
Marc Ratkovic
1056
https://doi.org/10.1017/S0003055422001022
 Published online by Cambridge University Press

by observations that are similar in some respects
(“homophily”) or different in some respects
(“heterophily”). For example, observations that are
geographically proximal may behave similarly (Ripley
1988; Ward and O’Loughlin 2002), actors may be con-
nected via a social network (Aronow a